In [2]:
import pandas as pd
from scipy.stats import ttest_ind

# Load the merged data
merged_data = pd.read_csv('../scripts/merged_data.csv')

# Convert 'Date' column to datetime format
merged_data['Date'] = pd.to_datetime(merged_data['Date'], infer_datetime_format=True)

# Set 'Date' as the index
merged_data.set_index('Date', inplace=True)

# Define significant events
significant_events = {
    '2008-09-15': 'Lehman Brothers Bankruptcy',
    '2020-03-11': 'WHO Declares COVID-19 a Pandemic',
    '2022-02-24': 'Russia Invades Ukraine'
}

# Function to perform t-test for impact analysis
def event_impact_test(event_date, window=30):
    event_date = pd.to_datetime(event_date)
    pre_event_window = merged_data.loc[event_date - pd.Timedelta(days=window):event_date]
    post_event_window = merged_data.loc[event_date:event_date + pd.Timedelta(days=window)]
    
    t_stat, p_value = ttest_ind(pre_event_window['Price'], post_event_window['Price'], nan_policy='omit')
    return t_stat, p_value

# Analyze the impact of each event
impact_results = {}
for date, event in significant_events.items():
    t_stat, p_value = event_impact_test(date)
    impact_results[event] = {'t_stat': t_stat, 'p_value': p_value}

# Display impact results
impact_results


/tmp/ipykernel_20592/1782906188.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  merged_data['Date'] = pd.to_datetime(merged_data['Date'], infer_datetime_format=True)


{'Lehman Brothers Bankruptcy': {'t_stat': np.float64(6.523362382132846),
  'p_value': np.float64(7.013699048678547e-08)},
 'WHO Declares COVID-19 a Pandemic': {'t_stat': np.float64(15.390200818204281),
  'p_value': np.float64(2.524922801425046e-19)},
 'Russia Invades Ukraine': {'t_stat': np.float64(-9.69464258238087),
  'p_value': np.float64(2.1938218600646873e-12)}}

In [3]:
# Function to calculate price changes before, during, and after events
def event_price_changes(event_date, window=30):
    event_date = pd.to_datetime(event_date)
    pre_event = merged_data.loc[event_date - pd.Timedelta(days=window):event_date]['Price']
    post_event = merged_data.loc[event_date:event_date + pd.Timedelta(days=window)]['Price']
    
    pre_mean = pre_event.mean()
    post_mean = post_event.mean()
    change = ((post_mean - pre_mean) / pre_mean) * 100  # Percentage change
    
    return pre_mean, post_mean, change

# Analyze price changes for each event
price_changes_results = {}
for date, event in significant_events.items():
    pre_mean, post_mean, change = event_price_changes(date)
    price_changes_results[event] = {'Pre-event Mean': pre_mean, 'Post-event Mean': post_mean, 'Percentage Change': change}

# Display price changes results
price_changes_results


{'Lehman Brothers Bankruptcy': {'Pre-event Mean': np.float64(105.56428571428572),
  'Post-event Mean': np.float64(88.34347826086956),
  'Percentage Change': np.float64(-16.313099962637946)},
 'WHO Declares COVID-19 a Pandemic': {'Pre-event Mean': np.float64(52.21826086956521),
  'Post-event Mean': np.float64(24.358695652173918),
  'Percentage Change': np.float64(-53.35215067192885)},
 'Russia Invades Ukraine': {'Pre-event Mean': np.float64(95.48565217391305),
  'Post-event Mean': np.float64(115.88181818181819),
  'Percentage Change': np.float64(21.36045106625708)}}

In [4]:
# Summarize findings
insights = []

for event in significant_events.values():
    t_stat = impact_results[event]['t_stat']
    p_value = impact_results[event]['p_value']
    pre_mean = price_changes_results[event]['Pre-event Mean']
    post_mean = price_changes_results[event]['Post-event Mean']
    change = price_changes_results[event]['Percentage Change']
    
    summary = (f"Event: {event}\n"
               f"Pre-event Mean Price: {pre_mean:.2f}\n"
               f"Post-event Mean Price: {post_mean:.2f}\n"
               f"Percentage Change: {change:.2f}%\n"
               f"T-statistic: {t_stat:.2f}\n"
               f"P-value: {p_value:.4f}\n"
               f"{'-'*50}")
    
    insights.append(summary)

# Display insights
for insight in insights:
    print(insight)


Event: Lehman Brothers Bankruptcy
Pre-event Mean Price: 105.56
Post-event Mean Price: 88.34
Percentage Change: -16.31%
T-statistic: 6.52
P-value: 0.0000
--------------------------------------------------
Event: WHO Declares COVID-19 a Pandemic
Pre-event Mean Price: 52.22
Post-event Mean Price: 24.36
Percentage Change: -53.35%
T-statistic: 15.39
P-value: 0.0000
--------------------------------------------------
Event: Russia Invades Ukraine
Pre-event Mean Price: 95.49
Post-event Mean Price: 115.88
Percentage Change: 21.36%
T-statistic: -9.69
P-value: 0.0000
--------------------------------------------------
